In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import requests
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
pd.set_option('display.float_format', lambda x: '%.4f' %x, "display.precision", 2)
np.set_printoptions(suppress=True)

In [2]:
index_code = pd.read_excel('indices.xlsx', engine='openpyxl')
index_code = index_code.iloc[:20, ]
index_code

,INDEX,CODE
0,BSE 500,BSE500
1,BSE AUTO,SI1900
2,BSE BANKEX,SIBANK
3,BSE Basic Materials,SPBSBMIP
4,BSE CAPITAL GOODS,SI0200
5,BSE Consumer Discretionary Goods Services,SPBSCDIP
6,BSE CONSUMER DURABLES,SI0400
7,BSE Energy,SPBSENIP
8,BSE Fast Moving Consumer Goods,SI0600
9,BSE Finance,SPBSFIIP


# Approach 1 - Rate of Change

In [3]:
def buy_sell_dates(dates, period):
    '''
    Calculates buy and sell dates based on defined period
    '''
    buy_dates = []
    sell_dates = []
    for i in range(0, len(dates), period):
        buy_dates.append(dates[i])
    for i in range(period, len(dates), period):
        sell_dates.append(dates[i])
    if(len(buy_dates)>len(sell_dates)):
        del buy_dates[-1]
    return buy_dates, sell_dates

def highlight_max(data, color='yellow'):
    '''
    Highlights the maximum value in a Series or DataFrame in yellow
    '''
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)
    
def calculate_trade_profits(period_returns, buy_dates, sell_dates):
    '''
    With buy and sell dates as inputs, this function looks up the period returns
    It then computes the trade return of doing buy and sell trades
    It creates a pandast dataframe with trade profits by trade number and 
    number of sectors. The sectors automatically vary from 1 sector to everything.
    '''
    trade_profits = np.zeros((len(buy_dates), period_returns.shape[1]))
    for a in range(len(buy_dates)):
        returns = period_returns.T[buy_dates[a]].sort_values(ascending=False)
        for i in range(len(returns)):
            sector_list = returns.index[:i+1]            
            trade_profit = period_returns.loc[sell_dates[a], sector_list].mean() 
            trade_profits[a, i] = trade_profit
    columns = [str(x+1)+' Sectors' for x in range(trade_profits.shape[1])]
    index = ['Trade Number ' + str(x+1) for x in range(trade_profits.shape[0])]
    return pd.DataFrame(trade_profits, columns = columns, index = index)

This cell below loads data and calculates 30 days, 60 days and 90 days returns

In [4]:
indices = pd.read_csv('indices_price.csv', header = 0, index_col = 0)
indices = indices.drop('BSE500', axis=1)
# Computes 30, 60 and 90 days returns
ninety_rets = indices.pct_change(90).dropna().loc[:'23-October-2020',]
rets_index = ninety_rets.index.to_list()
thirty_rets = indices.pct_change(30).dropna().loc[rets_index[0]:'23-October-2020',]
sixty_rets = indices.pct_change(60).dropna().loc[rets_index[0]:'23-October-2020',]
# indices.info()

This step below defines buy and sell dates for each of the three strategies, 30 days, 60 days and 90 days apart

In [5]:
thirty_buy_dates, thirty_sell_dates = buy_sell_dates(rets_index, 30)
sixty_buy_dates, sixty_sell_dates = buy_sell_dates(rets_index, 60)
ninety_buy_dates, ninety_sell_dates = buy_sell_dates(rets_index, 90)

In [6]:
print(len(thirty_buy_dates), len(thirty_sell_dates))
print(len(sixty_buy_dates), len(sixty_sell_dates))
print(len(ninety_buy_dates), len(ninety_sell_dates))

78 78
39 39
26 26


# Execute trades and calculate trade profits

In [7]:
thirty_trade_profits = calculate_trade_profits(thirty_rets, thirty_buy_dates, thirty_sell_dates)
sixty_trade_profits = calculate_trade_profits(sixty_rets, sixty_buy_dates, sixty_sell_dates)
ninety_trade_profits = calculate_trade_profits(ninety_rets, ninety_buy_dates, ninety_sell_dates)

In [8]:
thirty_trade_profits.head(2)

,1 Sectors,2 Sectors,3 Sectors,4 Sectors,5 Sectors,6 Sectors,7 Sectors,8 Sectors,9 Sectors,10 Sectors,11 Sectors,12 Sectors,13 Sectors,14 Sectors,15 Sectors,16 Sectors,17 Sectors,18 Sectors,19 Sectors
Trade Number 1,0.0268,0.0213,0.0243,0.0193,0.0233,0.0110,0.0102,0.0046,0.0032,0.0026,-0.0029,-0.0070,-0.0006,0.0003,-0.0018,-0.0014,-0.0006,-0.0047,-0.0036
Trade Number 2,-0.1050,-0.0600,-0.0301,-0.0215,-0.0213,-0.0284,-0.0380,-0.0420,-0.0485,-0.0460,-0.0538,-0.0536,-0.0572,-0.0633,-0.0660,-0.0650,-0.0649,-0.0653,-0.0667


# Compute average geometric yearly profits and put the data in a table

In [9]:
results_rateofchange = pd.DataFrame({'30 Days':((((1+thirty_trade_profits).astype('object').product())**0.111)-1)*100,
             '60 Days':((((1+sixty_trade_profits).astype('object').product())**0.111)-1)*100,
             '90 Days':((((1+ninety_trade_profits).astype('object').product())**0.111)-1)*100})

In [10]:
# Average yearly returns based on each strategy
results_rateofchange.style.apply(highlight_max)

,30 Days,60 Days,90 Days
1 Sectors,10.96,14.25,14.20
2 Sectors,9.24,14.42,7.90
3 Sectors,11.78,12.47,7.54
4 Sectors,10.55,11.09,8.92
5 Sectors,10.45,10.83,8.86
6 Sectors,10.15,10.40,9.78
7 Sectors,9.06,9.25,8.44
8 Sectors,9.34,8.22,8.35
9 Sectors,8.86,7.16,7.28
10 Sectors,8.80,6.98,8.19


In [11]:
# Percentage of profitable trades by each strategy
profitable_trades = pd.DataFrame({'30 Days':thirty_trade_profits.gt(0).mean()*100,
             '60 Days':sixty_trade_profits.gt(0).mean()*100,
             '90 Days':ninety_trade_profits.gt(0).mean()*100})
# Average yearly returns based on each strategy
profitable_trades.style.apply(highlight_max)

,30 Days,60 Days,90 Days
1 Sectors,57.69,56.41,61.54
2 Sectors,57.69,66.67,50.00
3 Sectors,64.10,66.67,53.85
4 Sectors,64.10,69.23,50.00
5 Sectors,62.82,66.67,61.54
6 Sectors,62.82,61.54,65.38
7 Sectors,60.26,58.97,65.38
8 Sectors,60.26,56.41,65.38
9 Sectors,60.26,53.85,57.69
10 Sectors,61.54,56.41,61.54


In [12]:
# Return/Risk ratio of trading strategy
thirty_sharpe = (results_rateofchange['30 Days']/100)/(thirty_trade_profits.std()*np.sqrt(250/30))
sixty_sharpe = (results_rateofchange['60 Days']/100)/(sixty_trade_profits.std()*np.sqrt(250/60))
ninety_sharpe = (results_rateofchange['90 Days']/100)/(sixty_trade_profits.std()*np.sqrt(250/90))
return_to_risk = pd.DataFrame({'30 Days':thirty_sharpe,
             '60 Days':sixty_sharpe,
             '90 Days':ninety_sharpe})
# Return to Risk Ratio of each trading strategy
return_to_risk.style.apply(highlight_max)

,30 Days,60 Days,90 Days
1 Sectors,0.46,0.62,0.75
2 Sectors,0.45,0.61,0.41
3 Sectors,0.62,0.60,0.44
4 Sectors,0.60,0.59,0.58
5 Sectors,0.59,0.58,0.58
6 Sectors,0.58,0.56,0.65
7 Sectors,0.52,0.51,0.57
8 Sectors,0.53,0.47,0.59
9 Sectors,0.50,0.41,0.51
10 Sectors,0.49,0.41,0.58


### Where to invest today based on Rate of Change

In [13]:
indices = pd.read_csv('indices_price.csv', header = 0, index_col = 0)
indices = indices.drop('BSE500', axis=1)
# Computes 30, 60 and 90 days returns
ninety_rets = indices.pct_change(90).dropna()
thirty_rets = indices.pct_change(30).dropna()
sixty_rets = indices.pct_change(60).dropna()

In [14]:
thirty_rets.iloc[-1:, ].T.sort_values(by='25-March-2021', ascending=False)[:3]

,25-March-2021
SIPOWE,0.1001
SPBSUTIP,0.0768
SPBSBMIP,0.0731


In [15]:
sixty_rets.iloc[-1:, ].T.sort_values(by='25-March-2021', ascending=False)[:3]

,25-March-2021
SPBSBMIP,0.2140
SIPOWE,0.2035
SPBSIDIP,0.1867


In [16]:
ninety_rets.iloc[-1:, ].T.sort_values(by='25-March-2021', ascending=False)[:3]

,25-March-2021
SI1200,0.4293
SPBSIDIP,0.3963
SPBSBMIP,0.3533


In [17]:
index_code[(index_code.CODE=='SPBSBMIP')|(index_code.CODE=='SIPOWE')|
           (index_code.CODE=='SPBSIDIP')|(index_code.CODE=='SI1200')|
           (index_code.CODE=='SPBSUTIP')]

,INDEX,CODE
3,BSE Basic Materials,SPBSBMIP
11,BSE Industrials,SPBSIDIP
13,BSE METAL,SI1200
15,BSE POWER,SIPOWE
19,BSE Utilities,SPBSUTIP
